# run_and_plot_multithreaded_mst_computation_performance

In [4]:
ls ../tp1

Makefile             fastsync.py*         sync.log
README               grafo.cpp            test/
borrar_logs.sh*      grafo.h              tp1_grafo.cpp
exp/                 primera_entrega/     tp1_grafo_old.cpp
fastfiles            primera_entrega.zip  watch.sh*


In [ ]:
from subprocess import Popen, PIPE

def flatten_list_of_strings(args):
    output = ''
    for s in args:
        output += s
        output += ' '
    return output

def str_to_num(s):
    if '.' in s:
        return float(s)
    else:
        return int(s)

class ExpRunner():

    def __init__(self, filter_name, input_image_path, implementation, output_image_dir, verbose=False):
        self.tp_exec = '../build/tp2'
        self.filter_name = filter_name
        self.exp_results = {'filter_name': filter_name, 'implementation': implementation}
        self.run_args = [self.tp_exec, 
                         filter_name, 
                         input_image_path, 
                         "-i", implementation, 
                         "-o", output_image_dir]
        self.verbose = verbose
        
    def exp_keys(self):
        '''
        ["Comienzo",
                "Fin",
                "# de ciclos insumidos totales",  
                "# de ciclos insumidos por llamada"]
        '''
        return ["# de ciclos insumidos totales"] # por ahora solo nos interesa esta

    def run_executable(self,run_args):
        p = Popen(run_args, stdout=PIPE, bufsize=1)
        with p.stdout:
            for line in iter(p.stdout.readline, b''):
                line = line.decode("utf-8")
                try:
                    key, val = line[:-1].split(':')
                    key = key.strip(" ")
                    if(key in self.exp_keys()):
                        self.exp_results[key] = str_to_num(val)
                except:
                    pass
        p.wait() # wait for the subprocess to exit

    def print_help(self):
        self.run_executable([self.tp_exec, "-h"])
        return
            
    def run(self):
        if self.verbose:
            print("Run command:")
            print(flatten_list_of_strings(self.run_args))
            print
        self.run_executable(self.run_args)
        return dict(self.exp_results) # esto es para copiar el resultado

### Cargar varias exp en un dataframe

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np

def get_exps(exp_runner, sample_size, z_score_threshold=2.5):

    # correr exps
    exps = pd.DataFrame([exp_runner.run() for i in range(sample_size)])

    # descartar outliers
    # exps = exps[stats.zscore(np.abs(exps['# de ciclos insumidos totales'])) < z_score_threshold]
    
    return exps

### correr exps :D

In [ ]:
filter_implementations_dir_path = "../filters/Implementaciones{}/".format(filter_name) 

mer=ExpRunner(filter_name = filter_name,
        input_image_path=img_path,
        implementation = "asm", 
        output_image_dir = "./exp_output_images/",
        verbose=False)

# exps = get_exps(mer, 30, z_score_threshold=2.5)
exps = get_exps(mer, qty_exps, z_score_threshold=2.5)


### Cuanto varia el tiempo de varias ejecuciones del mismo programa?